In [24]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler

In [20]:
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'lhl_student' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'lhl_student' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD
POSTGRES_DBNAME = 'mid_term_project' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
# Create the connection
cnx = create_engine(postgres_str)

In [3]:
flights = pd.read_sql_query('''SELECT fl_date, mkt_unique_carrier, op_unique_carrier, tail_num, op_carrier_fl_num, origin_airport_id,
origin, origin_city_name, dest_airport_id, dest, dest_city_name, dep_delay, distance, diverted FROM flights LIMIT 100000''', cnx)

In [4]:
flights.head(5)

,fl_date,mkt_unique_carrier,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,dep_delay,distance,diverted
0,2018-10-28,WN,WN,N212WN,3613,10800,BUR,"Burbank, CA",14771,SFO,"San Francisco, CA",8.0,326.0,0.0
1,2018-10-28,WN,WN,N7854B,5466,10800,BUR,"Burbank, CA",14771,SFO,"San Francisco, CA",34.0,326.0,0.0
2,2018-10-28,WN,WN,N210WN,5735,10800,BUR,"Burbank, CA",14771,SFO,"San Francisco, CA",15.0,326.0,0.0
3,2018-10-28,WN,WN,N968WN,1983,10800,BUR,"Burbank, CA",14831,SJC,"San Jose, CA",12.0,296.0,0.0
4,2018-10-28,WN,WN,N424WN,2592,10800,BUR,"Burbank, CA",14831,SJC,"San Jose, CA",-4.0,296.0,0.0


In [5]:
planes = pd.get_dummies(flights['tail_num'], prefix='tail_num', drop_first=True)
market_carriers = pd.get_dummies(flights['mkt_unique_carrier'], prefix='mkt_unique_carrier', drop_first=True)
op_carriers = pd.get_dummies(flights['op_unique_carrier'], prefix='op_unique_carrier', drop_first=True)
origins = pd.get_dummies(flights['origin_airport_id'], prefix='origin_airport_id', drop_first=True)
dests = pd.get_dummies(flights['dest_airport_id'], prefix='dest_airport_id', drop_first=True)
flight = pd.get_dummies(flights['op_carrier_fl_num'], prefix='fl_num', drop_first=True)
flights['route'] = flights['origin_airport_id'].astype(str) + '-' +  flights['dest_airport_id'].astype(str)
routes = pd.get_dummies(flights['route'], prefix='route', drop_first=True)
scaler = StandardScaler()

In [ ]:
flights_num = flights.copy()
flights_num.drop(flights_num[flights_num['diverted'] != 0.0].index, inplace=True)
numeric = flights_num[['dep_delay', 'distance']].copy()
numeric = pd.DataFrame(scaler.fit_transform(numeric), columns=['dep_delay_scaled','distance_scaled'])
flights_num = flights_num.join(numeric).join(routes).join(market_carriers).join(op_carriers).join(flight).join(planes)
flights_num.drop(columns=['tail_num', 'mkt_unique_carrier', 'op_unique_carrier', 'origin', 'origin_city_name', 'dest', 'dest_city_name', 'diverted', 'dep_delay', 'distance', 'route', 'origin_airport_id', 'dest_airport_id'], inplace=True)

In [ ]:
full = pd.read_csv('dfversion1.csv')

In [ ]:
#combined = pd.read_sql_query('''SELECT DISTINCT fl_date, mkt_unique_carrier, op_unique_carrier, tail_num, op_carrier_fl_num, flights.origin_airport_id, flights.origin, flights.origin_city_name, flights.dest_airport_id, flights.dest, flights.dest_city_name, dep_delay, flights.distance, diverted, departures_scheduled, departures_performed, seats, aircraft_group, aircraft_type, passengers, freight, mail, region FROM flights LEFT JOIN passengers ON flights.op_unique_carrier = passengers.unique_carrier LIMIT 100;''',cnx)

In [ ]:
#combined.head(5)

In [ ]:
passengers = pd.read_sql_query('''SELECT * FROM passengers LIMIT 100000;''',cnx)

In [ ]:
passengers.head(5)

In [ ]:
carriers = pd.get_dummies(passengers['unique_carrier'], drop_first=True)

In [ ]:
carriers

In [ ]:
market_carriers

In [ ]:
op_carriers

In [3]:
# pass2 = pd.read_sql_query('''
# SELECT departures_scheduled, departures_performed, seats, passengers, freight, mail, aircraft_group, aircraft_type, unique_carrier, region, origin_airport_id, dest_airport_id, CONCAT(origin_airport_id, '-', dest_airport_id) AS route
# FROM passengers
# LIMIT 1000;
# ''', cnx)

# fli_month = pd.read_sql_query('''
# SELECT op_unique_carrier, MAX(origin_airport_id) AS origin_airport_id, MAX(dest_airport_id) AS dest_airport_id, AVG(dep_delay) AS avg_dep_delay, MAX(distance) AS distance, CONCAT(origin_airport_id, '-', dest_airport_id) AS route, SUBSTRING(fl_date, 6, 2) AS month, SUBSTRING(fl_date, 1, 4) AS year
# FROM flights
# WHERE diverted != 1.0
# GROUP BY route, month, year, op_unique_carrier
# LIMIT 1000;
# ''', cnx)

In [6]:
fli_month.head(5)

,max,op_unique_carrier,max,max,avg,max,route,month,year
0,2018-01-31,DL,10135,10397,19.965517,692.0,10135-10397,01,2018
1,2018-01-31,EV,10135,10397,3.000000,692.0,10135-10397,01,2018
2,2019-01-31,9E,10135,10397,9.365854,692.0,10135-10397,01,2019
3,2019-01-31,DL,10135,10397,3.423077,692.0,10135-10397,01,2019
4,2018-02-26,9E,10135,10397,8.600000,692.0,10135-10397,02,2018


In [23]:
# comb2 = pd.read_sql_query('''
# SELECT op_unique_carrier, AVG(dep_delay) AS avg_dep_delay, CONCAT(flights.origin_airport_id, '-', flights.dest_airport_id) AS route, SUBSTRING(fl_date, 6, 2) AS month, SUBSTRING(fl_date, 1, 4) AS year
# INTO flight_subset
# FROM flights
# WHERE diverted != 1.0
# GROUP BY route, month, year, op_unique_carrier
# LIMIT 1000;

# SELECT departures_scheduled, departures_performed, seats, passengers, freight, mail, aircraft_group, aircraft_type, unique_carrier, region, origin_airport_id, dest_airport_id, CONCAT(origin_airport_id, '-', dest_airport_id) AS route, month, year, distance
# INTO passenger_subset
# FROM passengers
# LIMIT 1000;

# SELECT * FROM flight_subset LEFT JOIN passenger_subset ON (flight_subset.route = passenger_subset.route AND op_unique_carrier = unique_carrier AND flight_subset.month = passenger_subset.month AND flight_subset.year = passenger_subset.year)
# ''',cnx)

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "("
LINE 2: ...id, '-', flights.dest_airport_id) AS route, BIGINT(SUBSTRING...
                                                             ^

[SQL: 
SELECT op_unique_carrier, AVG(dep_delay) AS avg_dep_delay, CONCAT(flights.origin_airport_id, '-', flights.dest_airport_id) AS route, BIGINT(SUBSTRING(fl_date, 6, 2)) AS month, BIGINT(SUBSTRING(fl_date, 1, 4)) AS year
INTO flight_subset
FROM flights
WHERE diverted != 1.0
GROUP BY route, month, year, op_unique_carrier
LIMIT 1000;

SELECT departures_scheduled, departures_performed, seats, passengers, freight, mail, aircraft_group, aircraft_type, unique_carrier, region, origin_airport_id, dest_airport_id, CONCAT(origin_airport_id, '-', dest_airport_id) AS route, month, year, distance
INTO passenger_subset
FROM passengers
LIMIT 1000;

SELECT * FROM flight_subset LEFT JOIN passenger_subset ON (flight_subset.route = passenger_subset.route AND op_unique_carrier = unique_carri

In [22]:
comb = pd.read_sql_query('''
SELECT op_unique_carrier, AVG(dep_delay) AS avg_dep_delay, CONCAT(flights.origin_airport_id, '-', flights.dest_airport_id) AS route, SUBSTRING(fl_date, 6, 2) AS month, SUBSTRING(fl_date, 1, 4) AS year
INTO flight_subset
FROM flights
WHERE diverted != 1.0
GROUP BY route, month, year, op_unique_carrier
LIMIT 1000;

SELECT departures_scheduled, departures_performed, seats, passengers, freight, mail, aircraft_group, aircraft_type, unique_carrier, region, origin_airport_id, dest_airport_id, CONCAT(origin_airport_id, '-', dest_airport_id) AS route, month, year, distance
INTO passenger_subset
FROM passengers
LIMIT 1000;

SELECT passenger_subset.route, passenger_subset.month, passenger_subset.year, op_unique_carrier, avg_dep_delay, departures_scheduled, departures_performed, seats, passengers, freight, mail, aircraft_group, aircraft_type, region, passenger_subset.origin_airport_id, passenger_subset.dest_airport_id, distance 
FROM flight_subset LEFT JOIN passenger_subset ON (flight_subset.route = passenger_subset.route AND op_unique_carrier = unique_carrier AND flight_subset.month = passenger_subset.month AND flight_subset.year = passenger_subset.year)
GROUP BY passenger_subset.route, passenger_subset.month, passenger_subset.year, op_unique_carrier
LIMIT 1000;
''', cnx)

ProgrammingError: (psycopg2.errors.UndefinedFunction) operator does not exist: text = bigint
LINE 15: ..._carrier = unique_carrier AND flight_subset.month = passenge...
                                                              ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.

[SQL: 
SELECT op_unique_carrier, AVG(dep_delay) AS avg_dep_delay, CONCAT(flights.origin_airport_id, '-', flights.dest_airport_id) AS route, SUBSTRING(fl_date, 6, 2) AS month, SUBSTRING(fl_date, 1, 4) AS year
INTO flight_subset
FROM flights
WHERE diverted != 1.0
GROUP BY route, month, year, op_unique_carrier
LIMIT 1000;

SELECT departures_scheduled, departures_performed, seats, passengers, freight, mail, aircraft_group, aircraft_type, unique_carrier, region, origin_airport_id, dest_airport_id, CONCAT(origin_airport_id, '-', dest_airport_id) AS route, month, year, distance
INTO passenger_subset
FROM passengers
LIMIT 1000;

SELECT passenger_subset.route, p